In [ ]:
"""ESTABLISH A CONNECTION WITH THE DEVICE"""
import vna
import matplotlib.pyplot as plt

IP = "192.168.3.51"
timeout = 25e4 #[milliseconds]
max_npoints = 1e3  #if npoints > max_npoints the single scan is divided in more scans
instrument = vna.ManageVNA(IP,timeout,max_npoints)

In [ ]:
"""SET THE DESIRED PARAMETER BEFORE PERFORMING THE MEASUREMENTS"""
# Set [fmin,fmax] that is the frequency interval along which the scans are performed
fmin = 4e9
fmax = 8e9
# Set the power of the monotone waves sent into the port 1 to measure the selected matrix element (e.g. S21)
powerdBm = -15 #dBm
ifbw = 30 #Hz
# Set the number of equally spaced frequencies in the interval [fmin,fmax] sampled at each scan
npoints = 401
# Set the number of sweeps performed (that is the number of measurements whose average is returned)
navgs = 1

In [ ]:
"""
PERFORM THE MEASURE
We call the single_scan() method with the above parameters.
The method returns:
-freq: an array containing the sampled equally spaced frequencies.
-I and Q: respectively the real and the imaginary part of the matrix element values corresponding to the frequencies in "freq".
"""
instrument.set_IFBW(1e2)
freq, I, Q = instrument.single_scan(fmin,fmax,powerdBm,npoints,navgs,1)
S21dB = instrument.IQ_to_S21dB(I,Q)
# Here we calculate the matrix element in dB, form its real and imaginary parts
plt.plot(freq,S21dB)

In [ ]:
"""
Prepare VNA
"""
import time

instrument.reset()
time.sleep(3)
instrument.set_mode("NA")
instrument.set_port("S21")
instrument.query_reset()

In [ ]:
"""
Perform AND SAVE the measure
"""
from many_instruments import *
filename = "03_03_4-8Ghz_15pts.h5"
group = "Off"

acquire_S21(instrument,fmin,fmax,powerdBm,ifbw,npoints,navgs,filename,group,write_IQ=0)
